In [51]:
import json
import requests
import env
import pandas as pd
from datetime import datetime

class Data:
    METABASE_SESSION = ""
def getMetabaseSession(username, password):
    payload = {
        "username": username,
        "password": password
    }
    headers = {
        'content-type': "application/json",
    }
    response = requests.request("POST", env.METABASE_API_GET_SESSION, data=json.dumps(payload, separators=(',', ':')), headers=headers)
    Data.METABASE_SESSION = json.loads(response.text)['id']

def getMetaCards(card_ids):
    responses = []
    for card_id in card_ids:
        url = "http://metabase.jabama.com/api/card/" + str(card_id) + "/query"

        headers = {
            'Content-Type': "application/json",
            'X-Metabase-Session': Data.METABASE_SESSION,
            }

        responses.append([card_id, requests.request("POST", url, headers=headers)])
    return responses

def getDataFromPivotResponses(responses):
    responses_data = []
    
    for response in responses:
        cols = json.loads(response[1].text)['data']['cols']
        rows = json.loads(response[1].text)['data']['rows']
        card_id = response[0]
        col_titles = []
        for col in cols:
            col_titles.append(col['name'])

        responses_data.append({
            "card_id": card_id,
            "columns": col_titles,
            "rows": rows
        })
    return responses_data

def transformPivotResponseToDataFrame(pivotResponse):
    job_audit_df = pd.DataFrame(
        pivotResponse.get("rows"),
        columns = pivotResponse.get("columns")
    )[
        [
            "id",
            "OrderId",
            "PlaceCategory",
            "WorkflowStepId",
            "previous_workflowstepid",
            "next_workflowstepid",
            "nth",
            "CreatedDate",
            "UpdatedDate"
        ]
    ]
    return job_audit_df

def extractPrebookOrders(job_audit_df):
    prebook_orders_list = []
    prebook_flag = 0
    for indx, job_audit in job_audit_df.iterrows():
        
        if job_audit_df["nth"].iloc[indx] == 1:
            if prebook_flag == 1:
                prebook_flag = 0
                prebook_orders_list.append(prebook_list)
            if job_audit_df["WorkflowStepId"].iloc[indx] == "init" and job_audit_df["next_workflowstepid"].iloc[indx] == "init":
                prebook_flag = 1
                prebook_list = []
        if prebook_flag == 1:
            prebook_list.append(job_audit)
    return prebook_orders_list
    
def extractInstantOrders(job_audit_df):
    instant_orders_list = []
    instant_flag = 0
    for indx, job_audit in job_audit_df.iterrows():     
        if job_audit_df["nth"].iloc[indx] == 1:
            if instant_flag == 1:
                instant_flag = 0
                instant_orders_list.append(instant_list)
            if job_audit_df["WorkflowStepId"].iloc[indx] == "init" and job_audit_df["next_workflowstepid"].iloc[indx] != "init":
                instant_flag = 1
                instant_list = []
        if instant_flag == 1:
            instant_list.append(job_audit)
    return instant_orders_list
    
def splitOrderPathToSubPaths(prebook_batches):
    prebook_splitted_batches = []
    for prebook_batch in prebook_batches:
            being_splitted_list = []
            for job_audit in prebook_batch:
                being_splitted_list.append(job_audit)
                if job_audit["WorkflowStepId"] == "paymenttimeout" and job_audit["next_workflowstepid"] == "init":
                    prebook_splitted_batches.append(being_splitted_list)
                    being_splitted_list = []
            prebook_splitted_batches.append(being_splitted_list)
                
    return prebook_splitted_batches

def calculateTimeToHostConclusion(prebook_splitted_batches):
    prebook_splitted_batches_with_THO = []
    for prebook_splitted_batch in prebook_splitted_batches:
        start_dt = prebook_splitted_batch[0]["CreatedDate"]
        if start_dt[22] == "+":
            start_dt = start_dt[:22] + '0' + start_dt[22:]
        if start_dt[21] == "+":
            start_dt = start_dt[:21] + '00' + start_dt[21:]
        if start_dt[19] == "+":
            start_dt = start_dt[:19] + '.000' + start_dt[19:]
            
        end_dt = prebook_splitted_batch[-1]["UpdatedDate"]
        if end_dt[22] == "+":
            end_dt = end_dt[:22] + '0' + end_dt[22:]
        if end_dt[21] == "+":
            end_dt = end_dt[:21] + '00' + end_dt[21:]
        if end_dt[19] == "+":
            end_dt = end_dt[:19] + '.000' + end_dt[19:]
  
        THO = datetime.fromisoformat(end_dt) - datetime.fromisoformat(start_dt)
        prebook_splitted_batches_with_THO.append({
            "sub_order": prebook_splitted_batch,
            "THO": THO
        })
    return prebook_splitted_batches_with_THO

def metabot_job():
    #Get and store the raw data of union of questions from Metabase.
    pivot_metacard_responses = getMetaCards(["1638"])
    pivot_responses_data = getDataFromPivotResponses(pivot_metacard_responses)
    job_audit_df = transformPivotResponseToDataFrame(pivot_responses_data[0])
    
    prebook_order_batches_list = extractPrebookOrders(job_audit_df)
    instant_order_batches_list = extractInstantOrders(job_audit_df)

    prebook_splitted_batches = splitOrderPathToSubPaths(prebook_order_batches_list)
    prebook_splitted_batches_with_THO = calculateTimeToHostConclusion(prebook_splitted_batches)
    print(prebook_splitted_batches_with_THO)

    
def metabase_get_session_job():
    getMetabaseSession(env.METABASE_USERNAME, env.METABASE_PASSWORD)

metabase_get_session_job()
metabot_job()

[{'sub_order': [id                                               6115117
OrderId                                           765462
PlaceCategory                                        هتل
WorkflowStepId                                      init
previous_workflowstepid                             None
next_workflowstepid                                 init
nth                                                    1
CreatedDate                2020-06-09T22:47:32.637+03:30
UpdatedDate                2020-06-09T22:47:32.637+03:30
Name: 10, dtype: object, id                                               6115118
OrderId                                           765462
PlaceCategory                                        هتل
WorkflowStepId                                      init
previous_workflowstepid                             init
next_workflowstepid                                 init
nth                                                    2
CreatedDate                2020-06-09T22:47:32.